In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
# There are lots of unnecessary imports. I just copy and paste them all because I'm lazy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import display
from copy import deepcopy
import random
from time import time
import multiprocessing

# Set the display options for printing pandas dataframes
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 13)
pd.set_option('display.width', 250)




def sum_of_col(col, args):
    ''' 
    The method we want to apply to our dataframe of random numbers in parallel 
    
    col: pd.Series, a column from the dataframe
    args: tuple, some tuple of arguments you may wish to use 
    
    '''
    return col.sum()
 
def _apply_df(args):
    ''' 
    args: tuple
    df: pd.DataFrame
    func: The method we want to apply
    args: dict of arguments
    '''
    # Extract the variables from the tuple
    df, func, kwargs = args
    
    axis = kwargs.pop('axis')
    axis =[axis]
    return df.apply(func, args=axis)

def apply_by_multiprocessing(df, func, **kwargs):
    #print('Check 1', kwargs)
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(result)
    

    
if __name__ == '__main__':
    
        
    random_number_df = pd.DataFrame(np.random.randint(0,100,size=(100, 100)), columns=list(num for num in range(0,100)))
    print('random_number_df')
    print(random_number_df)
    
    
    times_df = pd.DataFrame(columns=['for loop', 'multiprocessing'])
    
    multiproc_time = pd.Series()
    i=0

    
    # Sum all columns with multiprocessing
    while i<10:

        ta = time()

        result = apply_by_multiprocessing(random_number_df, sum_of_col, axis=0, workers=1)
        #print(result)
        
        tb = time()
        current_time = tb - ta

        multiproc_time.set_value(i, current_time)

        i+=1
        

    times_df['multiprocessing'] = multiproc_time
    
    
    print('multiproc done')
    print(times_df)    
        
    #question_confidence.T.apply(sum_of_col, axis= 0)

    
    i=0

    singleproc_time = pd.Series()
    while i<10:

        ta = time()

        random_number_sum = pd.Series()
        for col in random_number_df:
            random_number_sum[str(col)] = random_number_df[col].sum()

        tb = time()
        current_time = tb - ta

        singleproc_time.set_value(i, current_time)
        #times_df['for loop'].append([], ignore_index=True)

        i+=1


    times_df['for loop'] = singleproc_time

    '''

    def  get(x):
        return x*x

    if __name__ == '__main__':
        pool = Pool(processes=3)              # start 4 worker processes
        result = pool.apply_async(f, [10])    # evaluate "f(10)" asynchronously
        print result.get(timeout=1)           # prints "100" unless your computer is *very* slow
        print pool.map(f, range(10)) 
        '''    

    print(times_df)

In [ ]:

random_number_df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
for col in random_number_df:
    print(col)